# 🎻 PyKwant Tutorial: Financial Instruments & Pricing
This notebook focuses on `pykwant.instruments`, the module dealing with financial products (like Bonds) and their valuation.

Unlike object-oriented libraries where a Bond "knows" how to price itself (e.g., `bond.price()`), **PyKwant** separates state from logic:

* **State**: Immutable Data Structures (e.g., `FixedRateBond`).

* **Logic**: Pure Functions that transform data (e.g., `generate_cash_flows`, `price_instrument`).

## 1. Setup and Imports

In [1]:
from datetime import date
from pykwant import instruments, dates, rates

# Helper to inspect cash flows
def print_flows(flows):
    print(f"{'Type':<10} | {'Date':<12} | {'Amount':>8}")
    print("-" * 35)
    for cf in flows:
        print(f"{cf.type:<10} | {cf.payment_date} | {cf.amount:>8.2f}")

## 2. Defining an Instrument (Immutable Data)
We define a **Fixed Rate Bond**. Notice that we inject the dependencies (calendar, day count) into the data structure, but the structure itself doesn't contain pricing logic.

**Scenario**:

* Face Value: 100.0

* Coupon: 5% (Annual)

* Start: Jan 1, 2025

* Maturity: Jan 1, 2028 (3 Years)

In [2]:
# 1. Define the parameters
bond = instruments.FixedRateBond(
    face_value=instruments.Money(100.0),
    coupon_rate=0.05,
    start_date=date(2025, 1, 1),
    maturity_date=date(2028, 1, 1),
    frequency_months=12,
    day_count=dates.thirty_360,
    calendar=dates.Calendar(holidays=frozenset()) # No holidays for simplicity
)

print(f"Instrument Created: {bond}")

Instrument Created: FixedRateBond(face_value=100.0, coupon_rate=0.05, start_date=datetime.date(2025, 1, 1), maturity_date=datetime.date(2028, 1, 1), frequency_months=12, day_count=<function thirty_360 at 0x0000021ACE1BBD70>, calendar=Calendar(holidays=frozenset(), weekends=(5, 6)), rolling=<function modified_following at 0x0000021ACE1BB950>)


## 3. Generating Cash Flows
The function `generate_cash_flows` is a pure function that takes a bond and returns a list of `CashFlow` objects. It handles the schedule generation and coupon calculation internally.

In [3]:
# Generate the deterministic flows
flows = instruments.generate_cash_flows(bond)

print("\n--- Asset Cash Flows ---")
print_flows(flows)

# Expected:
# 3 Coupons of 5.0
# 1 Principal repayment of 100.0 at maturity


--- Asset Cash Flows ---
Type       | Date         |   Amount
-----------------------------------
coupon     | 2026-01-01 |     5.00
coupon     | 2027-01-01 |     5.00
coupon     | 2028-01-03 |     5.03
principal  | 2028-01-03 |   100.00


## 4. Pricing the Instrument (Dirty Price)
Pricing requires a **Yield Curve**. In PyKwant, pricing is simply mapping the discounting function over the cash flows and summing the results.

We'll create a flat yield curve (5% continuous rate) using the `rates` module to price the bond. Since the bond pays 5% and the market rate is 5%, it should price near Par (100.0).

In [4]:
# 1. Create a Market Curve (Flat 5%)
# We use a lambda for demonstration or the rates factory
# Let's build a simple curve where DF(t) = e^(-0.05 * t)
ref_date = date(2025, 1, 1)

def flat_curve(d: date) -> float:
    t = dates.act_365(ref_date, d)
    return rates.compound_factor(-0.05, t, frequency=0)

# 2. Valuation Date
valuation_date = date(2025, 1, 1)

# 3. Calculate NPV (Dirty Price)
dirty_price = instruments.price_instrument(bond, flat_curve, valuation_date)

print(f"\n--- Valuation at {valuation_date} ---")
print(f"Market Rate: 5% Flat")
print(f"Dirty Price: {dirty_price:.4f}")


--- Valuation at 2025-01-01 ---
Market Rate: 5% Flat
Dirty Price: 99.6538


## 5. Accrued Interest & Clean Price
Bond traders quote the **Clean Price**, which separates the interest accrued since the last coupon. `Clean Price = Dirty Price - Accrued Interest`.

Let's move the valuation date forward to **July 1, 2025** (exactly halfway through the first coupon period).

* **Accrued**: Should be roughly 2.5 (half of the 5.0 coupon).

* **Dirty Price**: Should be ~102.5 (Par + Accrued).

* **Clean Price**: Should be ~100.0 (Par).

In [5]:
val_date_mid = date(2025, 7, 1)

# Recalculate Dirty Price at new date
dirty_mid = instruments.price_instrument(bond, flat_curve, val_date_mid)

# Calculate Accrued Interest
accrued = instruments.accrued_interest(bond, val_date_mid)

# Calculate Clean Price
clean = instruments.clean_price(bond, flat_curve, val_date_mid)

print(f"\n--- Valuation at {val_date_mid} (6 Months later) ---")
print(f"Dirty Price:      {dirty_mid:.4f}")
print(f"Accrued Interest: {accrued:.4f}")
print(f"Clean Price:      {clean:.4f}")

# Verification
assert abs((clean + accrued) - dirty_mid) < 1e-9
print("\nCheck: Clean + Accrued == Dirty [OK]")


--- Valuation at 2025-07-01 (6 Months later) ---
Dirty Price:      99.6538
Accrued Interest: 2.5000
Clean Price:      97.1538

Check: Clean + Accrued == Dirty [OK]


## 6. Functional Composition (Advanced)
One of the benefits of `pykwant` is compatibility with functional libraries like `toolz`. The `price_instrument` function is internally implemented as a pipeline: `Instrument -> Flows -> Filter(Future) -> Map(PV) -> Sum`.

This allows us to easily inspect intermediate steps, for example, filtering only Principal flows.

In [6]:
from toolz import filter

# Get only principal flows
principal_flows = list(filter(lambda cf: cf.type == "principal", flows))

print("\n--- Functional Filtering (Principal Only) ---")
print_flows(principal_flows)


--- Functional Filtering (Principal Only) ---
Type       | Date         |   Amount
-----------------------------------
principal  | 2028-01-03 |   100.00
